Данные для обучения возьмем из проекта [opencorpora.org](http://opencorpora.org/?page=downloads)
Скачаем размеченный корпус со снятой омонимией без UNKN как `oc.xml`

Разберем XML:

In [2]:
from xml.etree import ElementTree as ET

with open("oc.xml", mode="r", encoding="utf8") as f:
    doc = ET.parse(f)

In [23]:
all_sentences = []  # изначально пустой список предложений

# необходимо добраться до тегов sentence и достать из них части токены и их части речи
for text in doc.getroot():  # .getroot() возвращает тег annotation
    # text хранит элемент <text>
    # достаём из него список параграфов
    paragraphs = text.find('paragraphs')
    for paragraph in paragraphs:
        for sentence in paragraph:
            tokens = sentence.find('tokens')

            # новое предложение, полученное из xml
            new_sentence = []
            all_sentences.append(new_sentence)
            for token in tokens:
                word = token.get('text')
                # part of speech, <g v="POS">
                g = token.find('tfr').find('v').find('l').find('g')
                pos = g.get('v')
                new_sentence.append((word, pos))

In [24]:
# список предложений
# каждое предложение — это список пар (слово, тэг)
all_sentences[:10]

[[('«', 'PNCT'),
  ('Школа', 'NOUN'),
  ('злословия', 'NOUN'),
  ('»', 'PNCT'),
  ('учит', 'VERB'),
  ('прикусить', 'INFN'),
  ('язык', 'NOUN')],
 [('Сохранится', 'VERB'),
  ('ли', 'PRCL'),
  ('градус', 'NOUN'),
  ('дискуссии', 'NOUN'),
  ('в', 'PREP'),
  ('новом', 'ADJF'),
  ('сезоне', 'NOUN'),
  ('?', 'PNCT')],
 [('Великолепная', 'ADJF'),
  ('«', 'PNCT'),
  ('Школа', 'NOUN'),
  ('злословия', 'NOUN'),
  ('»', 'PNCT'),
  ('вернулась', 'VERB'),
  ('в', 'PREP'),
  ('эфир', 'NOUN'),
  ('после', 'PREP'),
  ('летних', 'ADJF'),
  ('каникул', 'NOUN'),
  ('в', 'PREP'),
  ('новом', 'ADJF'),
  ('формате', 'NOUN'),
  ('.', 'PNCT')],
 [('Потом', 'ADVB'),
  ('проект', 'NOUN'),
  ('переехал', 'VERB'),
  ('с', 'PREP'),
  ('«', 'PNCT'),
  ('Культуры', 'NOUN'),
  ('»', 'PNCT'),
  ('на', 'PREP'),
  ('НТВ', 'NOUN'),
  ('.', 'PNCT')],
 [('Это', 'NPRO'),
  ('помимо', 'PREP'),
  ('явных', 'ADJF'),
  ('перемен', 'NOUN'),
  ('в', 'PREP'),
  ('виде', 'NOUN'),
  ('тут', 'ADVB'),
  ('же', 'PRCL'),
  ('появившихс

Необходимо научиться превращать слова в набор признаков:

In [26]:
# получить признаки слова по предложению и номеру слова
# признаки — это словарь, где названию признака соответствует значение
def get_features(sentence, word_index):
    word, pos = sentence[word_index]

    # добавьте сами другие признаки, включая признаки для
    # предыдущего и следующего слов
    features = {
        '[-2:]': word[-2:]  # последние две буквы слова
    }
    if word_index == 0:
        features['START'] = True  # признак начала предложения
    else:
        pass  # признаки для предыдущего слова

    return features

In [27]:
all_features = [
    [get_features(sentence, ind) for ind in range(len(sentence))]
    for sentence in all_sentences
]

In [29]:
all_features[:3]

[[{'[-2:]': '«', 'START': True},
  {'[-2:]': 'ла'},
  {'[-2:]': 'ия'},
  {'[-2:]': '»'},
  {'[-2:]': 'ит'},
  {'[-2:]': 'ть'},
  {'[-2:]': 'ык'}],
 [{'[-2:]': 'ся', 'START': True},
  {'[-2:]': 'ли'},
  {'[-2:]': 'ус'},
  {'[-2:]': 'ии'},
  {'[-2:]': 'в'},
  {'[-2:]': 'ом'},
  {'[-2:]': 'не'},
  {'[-2:]': '?'}],
 [{'[-2:]': 'ая', 'START': True},
  {'[-2:]': '«'},
  {'[-2:]': 'ла'},
  {'[-2:]': 'ия'},
  {'[-2:]': '»'},
  {'[-2:]': 'сь'},
  {'[-2:]': 'в'},
  {'[-2:]': 'ир'},
  {'[-2:]': 'ле'},
  {'[-2:]': 'их'},
  {'[-2:]': 'ул'},
  {'[-2:]': 'в'},
  {'[-2:]': 'ом'},
  {'[-2:]': 'те'},
  {'[-2:]': '.'}]]

In [31]:
all_pos = [
    [pos for word, pos in sentence]
    for sentence in all_sentences
]

all_pos[:3]

[['PNCT', 'NOUN', 'NOUN', 'PNCT', 'VERB', 'INFN', 'NOUN'],
 ['VERB', 'PRCL', 'NOUN', 'NOUN', 'PREP', 'ADJF', 'NOUN', 'PNCT'],
 ['ADJF',
  'PNCT',
  'NOUN',
  'NOUN',
  'PNCT',
  'VERB',
  'PREP',
  'NOUN',
  'PREP',
  'ADJF',
  'NOUN',
  'PREP',
  'ADJF',
  'NOUN',
  'PNCT']]

In [32]:
%%time

import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,  # штраф за переобучение
    c2=0.1,  #
    max_iterations=100,
    all_possible_transitions=True
)
# для обучения передаём признаки и соответствующие им части речи
crf.fit(all_features, all_pos)
# AttributeError — не обращаем внимание, это баг

CPU times: user 22.8 s, sys: 406 ms, total: 23.2 s
Wall time: 25.1 s


AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

# Теперь модель обучена!

In [35]:
test_sentence = [("Он", "?"), ("идёт", "?"), ("в", "?"), ("школу", "?")]
# создаём признаки так же, как в прошлый раз
test_features = [
    [get_features(test_sentence, ind) for ind in range(len(test_sentence))]
]

crf.predict(test_features)

[['NPRO', 'VERB', 'PREP', 'NOUN']]

# Оценка качества, смотрим, что именно есть внутри модели

In [36]:
# список весов перехода между классами (частями речи)

crf.transition_features_

{('PNCT', 'PNCT'): 0.719936,
 ('PNCT', 'NOUN'): 3.603717,
 ('PNCT', 'VERB'): -0.04198,
 ('PNCT', 'INFN'): -2.018005,
 ('PNCT', 'PRCL'): -0.134438,
 ('PNCT', 'PREP'): -1.31654,
 ('PNCT', 'ADJF'): -1.138188,
 ('PNCT', 'ADVB'): 0.176762,
 ('PNCT', 'NPRO'): -0.612854,
 ('PNCT', 'PRTF'): 0.21973,
 ('PNCT', 'NUMB'): 0.494393,
 ('PNCT', 'PRED'): -1.267021,
 ('PNCT', 'COMP'): -1.346302,
 ('PNCT', 'CONJ'): 0.784601,
 ('PNCT', 'ADJS'): -0.833855,
 ('PNCT', 'GRND'): 0.701764,
 ('PNCT', 'LATN'): 2.336008,
 ('PNCT', 'PRTS'): -2.420539,
 ('PNCT', 'INTJ'): 0.332785,
 ('PNCT', 'SYMB'): -0.180698,
 ('PNCT', 'GREK'): -0.974961,
 ('PNCT', 'NUMR'): -1.771377,
 ('PNCT', 'ROMN'): -0.627618,
 ('PNCT', 'TIME'): -0.226183,
 ('PNCT', 'DATE'): -0.725408,
 ('PNCT', 'HANI'): 1.031947,
 ('NOUN', 'PNCT'): 0.839615,
 ('NOUN', 'NOUN'): 3.448366,
 ('NOUN', 'VERB'): -0.312437,
 ('NOUN', 'INFN'): -2.265759,
 ('NOUN', 'PRCL'): -1.138278,
 ('NOUN', 'PREP'): -0.973305,
 ('NOUN', 'ADJF'): -1.510243,
 ('NOUN', 'ADVB'): -0.706

In [41]:
# хочется найти самые большие веса и самые маленькие
# можно пользоваться классом Counter из стандартной библиотеки

from collections import Counter

# считает, сколько раз встретились элементы в перечислении
Counter([10, 20, 30, 10, 10, 10, 20])

# либо можно создать на основе уже готового словаря

cnt = Counter(crf.transition_features_)
for p in cnt.most_common(20):
    # найти максимальные значения
    print(p)

for p in cnt.most_common()[:20]:
    # найти максимальные значения, первые 20
    print(p)
print("---")
for p in cnt.most_common()[-20:]:
    # найти минимальные значения, последние 20
    print(p)

(('PREP', 'NOUN'), 7.486701)
(('ADJF', 'NOUN'), 7.390414)
(('NUMB', 'NOUN'), 6.366229)
(('NUMR', 'NOUN'), 6.133128)
(('VERB', 'NOUN'), 5.903229)
(('PRTF', 'NOUN'), 5.615523)
(('PRTS', 'NOUN'), 5.099541)
(('GRND', 'NOUN'), 5.093227)
(('CONJ', 'NOUN'), 4.671392)
(('PREP', 'LATN'), 4.529724)
(('INFN', 'NOUN'), 4.471478)
(('LATN', 'LATN'), 4.39063)
(('NPRO', 'NOUN'), 4.356182)
(('ADVB', 'NOUN'), 4.336339)
(('ROMN', 'NOUN'), 4.243924)
(('ADJS', 'NOUN'), 4.197572)
(('PRED', 'NOUN'), 4.102984)
(('SYMB', 'LATN'), 3.834136)
(('TIME', 'NOUN'), 3.758253)
(('PRCL', 'NOUN'), 3.679292)
(('PREP', 'NOUN'), 7.486701)
(('ADJF', 'NOUN'), 7.390414)
(('NUMB', 'NOUN'), 6.366229)
(('NUMR', 'NOUN'), 6.133128)
(('VERB', 'NOUN'), 5.903229)
(('PRTF', 'NOUN'), 5.615523)
(('PRTS', 'NOUN'), 5.099541)
(('GRND', 'NOUN'), 5.093227)
(('CONJ', 'NOUN'), 4.671392)
(('PREP', 'LATN'), 4.529724)
(('INFN', 'NOUN'), 4.471478)
(('LATN', 'LATN'), 4.39063)
(('NPRO', 'NOUN'), 4.356182)
(('ADVB', 'NOUN'), 4.336339)
(('ROMN', 'NOUN'

Можно аналогично делать с соответствием признаков и меток:

In [42]:
crf.state_features_  # возвращает веса между признаками и метками

{('[-2:]:«', 'PNCT'): 10.083587,
 ('START', 'PNCT'): 1.706367,
 ('START', 'NOUN'): 4.427813,
 ('START', 'VERB'): 0.029072,
 ('START', 'INFN'): -1.649741,
 ('START', 'PRCL'): 0.34799,
 ('START', 'PREP'): -1.043921,
 ('START', 'ADJF'): -0.544324,
 ('START', 'ADVB'): 1.098721,
 ('START', 'NPRO'): 0.022919,
 ('START', 'PRTF'): -1.028894,
 ('START', 'NUMB'): 0.489333,
 ('START', 'PRED'): -0.580038,
 ('START', 'COMP'): -0.258124,
 ('START', 'CONJ'): 1.199833,
 ('START', 'ADJS'): -0.430675,
 ('START', 'GRND'): -0.180897,
 ('START', 'LATN'): 2.976499,
 ('START', 'PRTS'): -0.817471,
 ('START', 'INTJ'): 0.994949,
 ('START', 'SYMB'): 0.87272,
 ('START', 'GREK'): 0.273643,
 ('START', 'NUMR'): -1.966776,
 ('START', 'ROMN'): 0.727917,
 ('START', 'TIME'): -0.763198,
 ('START', 'DATE'): 0.065439,
 ('START', 'HANI'): -1.39156,
 ('[-2:]:ла', 'NOUN'): 4.055794,
 ('[-2:]:ла', 'VERB'): 7.314528,
 ('[-2:]:ла', 'ADVB'): 3.548006,
 ('[-2:]:ия', 'NOUN'): 7.292606,
 ('[-2:]:»', 'PNCT'): 9.606268,
 ('[-2:]:ит', 

Определяем точность

In [43]:
# Сами определите точность классификации
# Пример, как будто мы определяли классы из y_true
# Определились как будто y_pred

y_true =['NOUN', 'ADJ', 'ADJ', 'VERB', 'NOUN']
y_pred =['NOUN', 'ADJ', 'VERB', 'NOUN', 'NOUN']

from sklearn import metrics

print(metrics.classification_report(y_true, y_pred, labels=['NOUN', 'ADJ', 'VERB'], digits=3))

# metrics.precision_score() — можно напрямую посчитать точность, например

              precision    recall  f1-score   support

        NOUN      0.667     1.000     0.800         2
         ADJ      1.000     0.500     0.667         2
        VERB      0.000     0.000     0.000         1

    accuracy                          0.600         5
   macro avg      0.556     0.500     0.489         5
weighted avg      0.667     0.600     0.587         5



Придумайте больше признаков, сделайте обучающий и тестовый корпусы, вычислите качество классификации. Дополнительно, подберите гиперпараметры для обучения crf.

Код: [https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html](https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html)